# Example of end-to-end pipeline on example data

## Import of ontoconnectlm packages

In [ ]:
import sys

sys.path.append("../")

from ontoconnectlm.classes_generator import ClassesGenerator    # Service 1
from ontoconnectlm.triplet_extractor import TripletExtractor    # Service 2
from ontoconnectlm.owl_generator import OwlGenerator            # Service 3
from ontoconnectlm.onto_enricher import OntoEnricher            # Service 4
from ontoconnectlm.onto_eval import Onto_Evaluator              # Service 5

## Loading of texts

Below are examples of French texts dealing with nuclear engineering requirements

In [ ]:
texts = [
    "#0001_Le condenseur principal doit recevoir les condensats provenant de réchauffeurs basse pression (B.P.), l'eau d'appoint à l'eau condensée, l'eau et la vapeur des purges de divers matériels et, s'il y a lieu, l'eau condensée provenant des condenseurs auxiliaires.",
    "#0002_Pour la réalisation de tout revêtement par soudage, le Titulaire doit soumettre à l'Entreprise pour accord, les dispositions prévues pour pallier les risques de fissuration à froid et de fissuration au réchauffage. Les défauts sous revêtement doivent être prohibés.",
    "#0003_Toutes les tubulures sont placées sur le plateau de serrage fixe.s",
    "#0004_Le code de dimensionnement de la platine du pied support (à l'interface entre le matériel et le Génie Civil ou l'ossature métallique) doit être le code de dimensionnement du matériel.",
    "#0005_Le condenseur principal doit assurer l'extraction des gaz incondensables de l'ensemble du circuit eau-vapeur à l'exclusion de ceux des condenseurs auxiliaires.",
    "#0006_Toute intervention, d'inspection en service, d'entretien et de nettoyage ou de remplacement de pièces d'usure, définie sur le matériel doit s'inscrire dans une fenêtre temporelle définie au CCTP."
]

## Service 1

In [ ]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(base_url="127.0.0.1:11434" , model="mistral-small3.1:24b") # Change with your llm instance

context_description = "Les textes portent sur des exigences formulées dans le domaine nucléaire."

generator = ClassesGenerator(
    llm = llm,
    context_description=context_description,
)

found_classes = generator.run(
    texts = texts,
    nb_classes = 6
)

display(found_classes)

## Service 2

In [ ]:
# Setting extraction based on schema generated at step 1
entity_types = found_classes

relation_types = [
    "IS_ASSOCIATED_WITH",

    "IS_COMPLIANT_WITH",

    "IS_DELIVERED_TO",

    "IS_DESCRIBED_IN",

    "IS_PROVIDING",
]


extractor = TripletExtractor(
    llm = llm,
    context_description=context_description,
    entity_types=entity_types,
    relation_types=relation_types
)

In [ ]:
triples = extractor.run(texts) 

In [ ]:
triples

In [ ]:
for i in range(0,len(triples)):
        for triple in triples[i]:
                print(triple["head"] , " (" , triple["head_type"] , ") " , triple["label"], triple["tail"] , " (" , triple["tail_type"] , ") ")
                # Générer une liste finale des sorties de l'ontologie

## Service 3 : Owl Generation

### Defining prerequisites for the ontology

Note : it is important for entity types to be in lowercases

In [ ]:
# Defining classes as capital letter form of entity type

entity_linker = {
    "other": "other"
}

for entity_type in entity_types:
    entity_linker[entity_type.lower()] = entity_type.capitalize()

entity_linker

### Optional (if you want to enforce some classes and properties in the ontology)

In [ ]:
required_classes = ["Actor", "Operation", "System"]
required_properties = {}
onto_uri = "http://smd#"

In [ ]:
config = {
    "entity_linker" : entity_linker,
    "additional_classes" : required_classes,
    "additional_properties" : required_properties
}

### Gathering all triples together

In [ ]:
all_triples = []
for sentence_triples in triples:
    for triple in sentence_triples:
        all_triples.append(triple)

In [ ]:
owl_generator = OwlGenerator(
    triples = all_triples,
    config= config,
    onto_uri = onto_uri
)

In [ ]:
result_ontology = owl_generator.create_ontology()

print(result_ontology)

## Service 4 : Ontology enricher from DBpedia and Wikidata

In [ ]:
onto_enricher = OntoEnricher(ontology_content=result_ontology)

enriched_ontology = onto_enricher.run_all_enrichments(enrichment_mode="classes")

In [ ]:
print(enriched_ontology)

### Optional : save ontology in file

In [ ]:
with open("enriched_ontology.owl" , 'w') as write_file:
    write_file.write(enriched_ontology)

## Service 5 : Ontology evaluation

In [ ]:
evaluator = Onto_Evaluator(ontology_content=enriched_ontology, 
                           ontology_uri = onto_uri)

### OOPS eval

In [ ]:
evaluator.eval_oops()

In [ ]:
evaluator.eval_fair()